
# Load Data, Drop Unecessary Information

In [63]:
import pandas as pd
import os

In [64]:
def create_complete_df():
    path_to_features  = '/content/drive/MyDrive/SOCData'

    list_of_dfs = []

    for csv_file in os.listdir(path_to_features):
        if csv_file.endswith(".csv") and csv_file != 'HWSD_DATA.csv':
            print(csv_file)
            df = pd.read_csv(os.path.join(path_to_features, csv_file))
            list_of_dfs.append(df)

    df = pd.concat(list_of_dfs, axis=0)
    return df

In [65]:
df = create_complete_df()

0.csv
1.csv


In [66]:
df

,system:index,AFRI1600,AFRI2100,ANDWI,AVI,AWEInsh,AWEIsh,B1,B10,B11,...,VARI,VI6T,VIBI,VIG,VgNIRBI,VrNIRBI,WI1,WI2,WRI,.geo
0,0,0.149063,0.344371,-0.212260,0.204699,0.126791,-0.360973,0.142480,287.075938,286.696038,...,-0.226988,0.805830,0.593608,-0.145814,-0.224955,-0.081875,-0.212943,-0.294912,0.698372,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1,0.128493,0.349047,-0.228201,0.206454,0.043941,-0.380266,0.132384,280.466259,280.466191,...,-0.232534,0.794399,0.542664,-0.149023,-0.237986,-0.092418,-0.220916,-0.297460,0.667642,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2,0.114117,0.351467,-0.233191,0.209286,-0.009739,-0.389533,0.132415,277.570303,277.593512,...,-0.221849,0.787871,0.530175,-0.139284,-0.237723,-0.102051,-0.218050,-0.283456,0.650535,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,3,0.127638,0.359274,-0.227560,0.206772,0.008566,-0.369403,0.129619,280.640067,280.592952,...,-0.228370,0.781420,0.566774,-0.143659,-0.242806,-0.102976,-0.214465,-0.275271,0.656328,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,4,0.127219,0.346265,-0.236327,0.207273,0.031360,-0.384046,0.131205,284.049542,283.873270,...,-0.243474,0.793904,0.553394,-0.155914,-0.248240,-0.096133,-0.234135,-0.307113,0.657217,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,45,0.256773,0.433491,-0.073551,0.187188,0.362525,-0.136723,0.188062,271.251923,271.730467,...,-0.136050,0.810301,0.034407,-0.079472,-0.143129,-0.064543,-0.021063,-0.063210,0.867531,"{""type"":""MultiPoint"",""coordinates"":[]}"
46,46,0.122983,0.343581,-0.206415,0.222997,0.103847,-0.381959,0.169202,276.219829,276.648642,...,-0.171326,0.824627,0.424061,-0.105502,-0.198272,-0.094951,-0.184866,-0.252130,0.689489,"{""type"":""MultiPoint"",""coordinates"":[]}"
47,47,0.128561,0.349216,-0.189565,0.212326,0.112682,-0.347290,0.169593,271.468434,271.968891,...,-0.165262,0.819601,0.716668,-0.100305,-0.186076,-0.087684,-0.165990,-0.224272,0.707556,"{""type"":""MultiPoint"",""coordinates"":[]}"
48,48,0.157961,0.349295,-0.143375,0.170742,0.220949,-0.199897,0.159222,271.656846,272.317001,...,-0.149907,0.787641,0.544234,-0.087554,-0.149499,-0.062803,-0.131128,-0.175602,0.772172,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [67]:
df.reset_index(inplace = True)

In [68]:
df.drop(['system:index', '.geo'], axis = 1, inplace = True)

In [69]:
df

,index,AFRI1600,AFRI2100,ANDWI,AVI,AWEInsh,AWEIsh,B1,B10,B11,...,UI,VARI,VI6T,VIBI,VIG,VgNIRBI,VrNIRBI,WI1,WI2,WRI
0,0,0.149063,0.344371,-0.212260,0.204699,0.126791,-0.360973,0.142480,287.075938,286.696038,...,-0.012548,-0.226988,0.805830,0.593608,-0.145814,-0.224955,-0.081875,-0.212943,-0.294912,0.698372
1,1,0.128493,0.349047,-0.228201,0.206454,0.043941,-0.380266,0.132384,280.466259,280.466191,...,-0.017876,-0.232534,0.794399,0.542664,-0.149023,-0.237986,-0.092418,-0.220916,-0.297460,0.667642
2,2,0.114117,0.351467,-0.233191,0.209286,-0.009739,-0.389533,0.132415,277.570303,277.593512,...,-0.020635,-0.221849,0.787871,0.530175,-0.139284,-0.237723,-0.102051,-0.218050,-0.283456,0.650535
3,3,0.127638,0.359274,-0.227560,0.206772,0.008566,-0.369403,0.129619,280.640067,280.592952,...,-0.029610,-0.228370,0.781420,0.566774,-0.143659,-0.242806,-0.102976,-0.214465,-0.275271,0.656328
4,4,0.127219,0.346265,-0.236327,0.207273,0.031360,-0.384046,0.131205,284.049542,283.873270,...,-0.014755,-0.243474,0.793904,0.553394,-0.155914,-0.248240,-0.096133,-0.234135,-0.307113,0.657217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,45,0.256773,0.433491,-0.073551,0.187188,0.362525,-0.136723,0.188062,271.251923,271.730467,...,-0.121045,-0.136050,0.810301,0.034407,-0.079472,-0.143129,-0.064543,-0.021063,-0.063210,0.867531
96,46,0.122983,0.343581,-0.206415,0.222997,0.103847,-0.381959,0.169202,276.219829,276.648642,...,-0.012673,-0.171326,0.824627,0.424061,-0.105502,-0.198272,-0.094951,-0.184866,-0.252130,0.689489
97,47,0.128561,0.349216,-0.189565,0.212326,0.112682,-0.347290,0.169593,271.468434,271.968891,...,-0.019327,-0.165262,0.819601,0.716668,-0.100305,-0.186076,-0.087684,-0.165990,-0.224272,0.707556
98,48,0.157961,0.349295,-0.143375,0.170742,0.220949,-0.199897,0.159222,271.656846,272.317001,...,-0.018508,-0.149907,0.787641,0.544234,-0.087554,-0.149499,-0.062803,-0.131128,-0.175602,0.772172


In [70]:
df.dropna()

,index,AFRI1600,AFRI2100,ANDWI,AVI,AWEInsh,AWEIsh,B1,B10,B11,...,UI,VARI,VI6T,VIBI,VIG,VgNIRBI,VrNIRBI,WI1,WI2,WRI
0,0,0.149063,0.344371,-0.212260,0.204699,0.126791,-0.360973,0.142480,287.075938,286.696038,...,-0.012548,-0.226988,0.805830,0.593608,-0.145814,-0.224955,-0.081875,-0.212943,-0.294912,0.698372
1,1,0.128493,0.349047,-0.228201,0.206454,0.043941,-0.380266,0.132384,280.466259,280.466191,...,-0.017876,-0.232534,0.794399,0.542664,-0.149023,-0.237986,-0.092418,-0.220916,-0.297460,0.667642
2,2,0.114117,0.351467,-0.233191,0.209286,-0.009739,-0.389533,0.132415,277.570303,277.593512,...,-0.020635,-0.221849,0.787871,0.530175,-0.139284,-0.237723,-0.102051,-0.218050,-0.283456,0.650535
3,3,0.127638,0.359274,-0.227560,0.206772,0.008566,-0.369403,0.129619,280.640067,280.592952,...,-0.029610,-0.228370,0.781420,0.566774,-0.143659,-0.242806,-0.102976,-0.214465,-0.275271,0.656328
4,4,0.127219,0.346265,-0.236327,0.207273,0.031360,-0.384046,0.131205,284.049542,283.873270,...,-0.014755,-0.243474,0.793904,0.553394,-0.155914,-0.248240,-0.096133,-0.234135,-0.307113,0.657217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,45,0.256773,0.433491,-0.073551,0.187188,0.362525,-0.136723,0.188062,271.251923,271.730467,...,-0.121045,-0.136050,0.810301,0.034407,-0.079472,-0.143129,-0.064543,-0.021063,-0.063210,0.867531
96,46,0.122983,0.343581,-0.206415,0.222997,0.103847,-0.381959,0.169202,276.219829,276.648642,...,-0.012673,-0.171326,0.824627,0.424061,-0.105502,-0.198272,-0.094951,-0.184866,-0.252130,0.689489
97,47,0.128561,0.349216,-0.189565,0.212326,0.112682,-0.347290,0.169593,271.468434,271.968891,...,-0.019327,-0.165262,0.819601,0.716668,-0.100305,-0.186076,-0.087684,-0.165990,-0.224272,0.707556
98,48,0.157961,0.349295,-0.143375,0.170742,0.220949,-0.199897,0.159222,271.656846,272.317001,...,-0.018508,-0.149907,0.787641,0.544234,-0.087554,-0.149499,-0.062803,-0.131128,-0.175602,0.772172


# Merge HWSD Data

In [71]:
HWSD = pd.read_csv('/content/drive/MyDrive/SOCData/HWSD_DATA.csv')
HWSD = HWSD[['MU_GLOBAL', 'S_OC']]
HWSD

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,MU_GLOBAL,S_OC
0,7001,NaN
1,7002,NaN
2,7003,NaN
3,7004,32.89
4,7005,NaN
...,...,...
48143,32049,0.13
48144,32049,NaN
48145,32050,NaN
48146,32050,0.13


In [72]:
new = pd.merge(df, HWSD, on="MU_GLOBAL")
new = new.dropna()

# Split Data

In [73]:
y = new.pop('S_OC')
X = new

In [75]:
assert 'S_OC' not in X.columns

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [77]:
assert len(X_train) == len(y_train)

In [78]:
assert len(X_test) == len(y_test)

# Random Forest Regressor

In [79]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X, y)

RandomForestRegressor()

# RMSE

In [86]:
from sklearn.metrics import mean_squared_log_error

round(mean_squared_log_error(rf.predict(X_test), y_test), 3)

0.0